## Bundling custom Python modules

This guide shows how to bundle custom Python modules in your Docker image.

Bundling modules can simplify your setup when you want to include custom libraries that require special credentials and such credentials are already configured locally.

In [1]:
# conda activate {env} doesn't work well here
# so we manually modify the path
PATH=$CONDA_PREFIX/envs/soopervisor/bin:$PATH

In [7]:
# TODO: this should download the directory
cd ../../tutorials/local-lib

In [8]:
pwd

/Users/Edu/dev/soopervisor/tutorials/local-lib


In [9]:
ls

__pycache__/            out.txt                 soopervisor.local.yaml
cluster/                pipeline.yaml           soopervisor.yaml
dist/                   requirements.lock.txt   tasks.py
lib/                    script.py


In [10]:
rm -rf cluster dist soopervisor.yaml

In [11]:
soopervisor add cluster --backend argo-workflows

================================= Loading DAG ==================================
No pipeline.cluster.yaml found, looking for pipeline.yaml instead
Found /Users/Edu/dev/soopervisor/tutorials/local-lib/pipeline.yaml. Loading...
= Adding /Users/Edu/dev/soopervisor/tutorials/local-lib/cluster/Dockerfile... ==
===================================== Done =====================================
Environment added, to export it:
	 $ soopervisor export cluster
To force execution of all tasks:
	 $ soopervisor export cluster --mode force



In [12]:
cp soopervisor.local.yaml soopervisor.yaml

In [13]:
cat soopervisor.yaml

cluster:
  backend: argo-workflows
  exclude: [output]
  repository: null


Note that the `Dockerfile` adds  `lib/` to the `PYTHONPATH` automatically:

In [14]:
cat cluster/Dockerfile

FROM condaforge/mambaforge:4.10.1-0

# find custom lib and included in PYTHONPATH



ENV PYTHONPATH lib/



COPY requirements.lock.txt project/requirements.lock.txt


RUN pip install --requirement project/requirements.lock.txt && rm -rf /root/.cache/pip/


COPY dist/* project/
WORKDIR /project/

# extract to get any config files at the root
RUN tar --strip-components=1 -zxvf *.tar.gz
RUN cp -r /project/ploomber/ /root/.ploomber/  || echo 'ploomber home does not exist'



In [18]:
soopervisor export cluster --skip-tests --ignore-git --mode force &> out.log

In [19]:
docker run local-lib:latest-default python -c "from functions import print_message; print_message()"

Hello from lib/


In [20]:
docker run local-lib:latest-default ploomber build

Loading pipeline...
Building task 'task': 100%|██████████| 2/2 [00:04<00:00,  2.30s/it]
name    Ran?      Elapsed (s)    Percentage
------  ------  -------------  ------------
script  True         4.56829     99.9511
task    True         0.002234     0.0488785
